In [1]:
import robin_stocks.robinhood as r
import pandas as pd
import datetime as dt
import os
import talib
import numpy as np
import yfinance as y
from src.storage.googlesheets import getSheet,appendTo,clearSheet
from iexfinance.stocks import get_historical_data
from datetime import datetime 
from datetime import date,timedelta
from iexfinance.refdata import get_symbols

def login():
    os.environ['IEX_TOKEN']="insert token"
    pd.set_option('display.max_columns', None)

    robin_user = os.environ.get("username")
    robin_pass = os.environ.get("password")
    r.login(username=robin_user,
             password=robin_pass,
             expiresIn=86400,
             by_sms=True)


def get_stocks():
    stocks = r.get_top_100()
    tickers = []
    for number in range(len(stocks)):
        for element in stocks[0]:
            if element == "symbol":
                tickers.append(stocks[number][element])
    return tickers


def get_stocks_from_file(file_name):
    with open(file_name) as reader:
        stock_list = reader.readlines()
        for n in range(len(stock_list)):
            stock_list[n] = stock_list[n].strip("\n")
        return stock_list


def filter_stocks_by_price(tickers, price_to_filter):
    tickers_under_price = {}
    for ticker in tickers:
        new_tickers = r.get_quotes(tickers)
        for new_ticker in new_tickers:
            if float(new_ticker["last_trade_price"]) <= price_to_filter:
                local_ticker = new_ticker["symbol"]
                tickers_under_price[local_ticker] = new_ticker["last_trade_price"]
    print(tickers_under_price)
    return tickers_under_price


def delta_probability_strike_price_finder(tickers_under_price, exp_date, call_or_put):
    delta_and_win_percent = {}
    for ticker in tickers_under_price:
        try:
            strike_price = str(round(float(tickers_under_price.get(ticker)), 1))
            while True:
                try:
                    option_data = r.find_options_by_expiration_and_strike(
                        ticker, exp_date, strike_price, call_or_put
                    )
                    if call_or_put == "put":
                        if (float(strike_price) % 0.5) == 0:
                            strike_price = str(round(float(strike_price) - 0.50, 1))
                            if float(option_data[0]["delta"]) >= -0.15:
                                break
                        else:
                            strike_price = str(round(float(strike_price) - 0.10, 1))
                    if call_or_put == "call":
                        if (float(strike_price) % 0.5) == 0:
                            strike_price = str(round(float(strike_price) + 0.50, 1))
                            if float(strike_price) >= float(
                                (5 * float(tickers_under_price.get(ticker)))
                            ):
                                break
                            if float(option_data[0]["delta"]) <= 0.3:
                                break
                        else:
                            strike_price = str(round(float(strike_price) + 0.10, 1))
                except IndexError:
                    if float(strike_price) <= 0.0:
                        break
                    continue
            print(option_data)
            delta_and_win_percent[ticker] = [
                option_data[0]["last_trade_price"],
                option_data[0]["delta"],
                option_data[0]["strike_price"],
                tickers_under_price[ticker],
            ]
        except AttributeError:
            print("could not get options")
        except IndexError:
            print("excluded from screening because price is too high")
    print(delta_and_win_percent)
    return delta_and_win_percent


def print_option_data(delta_and_win_percent):
    for stock in delta_and_win_percent:
        percent_win = round(
            (
                float(delta_and_win_percent[stock][0])
                / float(delta_and_win_percent[stock][3])
            )
            * 100,
            2,
        )
        probability = abs(round(float(delta_and_win_percent[stock][1]) * 100, 2))
        strike = round(float(delta_and_win_percent[stock][2]), 2)
        option_price = round(float(delta_and_win_percent[stock][0]), 2)
        price = round(float(delta_and_win_percent[stock][3]), 2)

        print(
            str(stock)
            + ":  Percent Win - "
            + str(percent_win)
            + "% Probability Lose - "
            + str(probability)
            + "% "
            + " Strike - "
            + str(strike)
            + " Option Price - "
            + str(option_price)
            + " Price - "
            + str(price)
        )



In [ ]:
login()
get_stocks_or_pull_stocks = input(
    "Would you like to pull stocks from a file or use Robinhood's top 100? (1 for pull from file 2 for top 100)"
)
if get_stocks_or_pull_stocks == "2":
    list_of_stocks = get_stocks()
elif get_stocks_or_pull_stocks == "1":
    list_of_stocks = get_stocks_from_file("tickers_to_search")
else:
    print("An Error Has Occurred")
price_to_filter_by = input(
    "What is the maximum share price for the stocks you are looking for?"
)
filtered_stocks = filter_stocks_by_price(list_of_stocks, float(price_to_filter_by))
strike_to_filter_by = input(
    "What is the date of expiry of the options you are looking for? (follow the format YYYY-MM-DD)"
)
calls_or_puts = input("Are you looking for calls or puts? (1 for calls 2 for puts)")
if calls_or_puts == "1":
    options_data = delta_probability_strike_price_finder(
        filtered_stocks, strike_to_filter_by, "call"
    )
elif calls_or_puts == "2":
    options_data = delta_probability_strike_price_finder(
        filtered_stocks, strike_to_filter_by, "put"
    )
else:
    print("An Error Has Occurred")
print_option_data(options_data)
input("Press enter to quit")

Would you like to pull stocks from a file or use Robinhood's top 100? (1 for pull from file 2 for top 100)2
What is the maximum share price for the stocks you are looking for?300
{'AAPL': '147.909900', 'AMC': '42.945000', 'SNDL': '0.780600', 'F': '13.034400', 'NIO': '39.075000', 'DIS': '178.230000', 'PFE': '47.640000', 'AAL': '20.330000', 'PLUG': '26.550000', 'HOOD': '48.670000', 'CCL': '24.060100', 'NOK': '6.055000', 'ACB': '7.160000', 'GPRO': '10.040000', 'ZOM': '0.701700', 'LCID': '21.880000', 'NAKD': '0.737800', 'BABA': '167.060000', 'DAL': '41.310000', 'PLTR': '24.870000', 'SNAP': '73.495000', 'GME': '206.710000', 'BAC': '42.630000', 'OGI': '2.675000', 'BB': '10.995000', 'COIN': '252.427100', 'NVDA': '219.936800', 'TLRY': '13.620000', 'SBUX': '114.814700', 'SPCE': '26.210000', 'AMD': '107.960000', 'CGC': '17.800100', 'T': '27.310000', 'TWTR': '63.740000', 'KO': '55.630000', 'FCEL': '6.415000', 'NCLH': '25.990000', 'IDEX': '2.345000', 'GM': '48.820000', 'WKHS': '9.770000', 'GE': '1

Loading Market Data Loading Market Data Loading Market Data Loading Market Data [{'chain_id': '2c081432-2180-4c5f-ae36-3587554758fc', 'chain_symbol': 'NIO', 'created_at': '2021-07-22T00:07:58.391588Z', 'expiration_date': '2021-09-03', 'id': 'cae11791-23a0-4253-bb02-18eba1bed19d', 'issue_date': '2021-07-22', 'min_ticks': {'above_tick': '0.05', 'below_tick': '0.01', 'cutoff_price': '3.00'}, 'rhs_tradability': 'untradable', 'state': 'active', 'strike_price': '41.0000', 'tradability': 'tradable', 'type': 'call', 'updated_at': '2021-07-22T00:07:58.391593Z', 'url': 'https://api.robinhood.com/options/instruments/cae11791-23a0-4253-bb02-18eba1bed19d/', 'sellout_datetime': '2021-09-03T19:00:00+00:00', 'long_strategy_code': 'cae11791-23a0-4253-bb02-18eba1bed19d_L1', 'short_strategy_code': 'cae11791-23a0-4253-bb02-18eba1bed19d_S1', 'adjusted_mark_price': '0.580000', 'ask_price': '0.590000', 'ask_size': 2, 'bid_price': '0.570000', 'bid_size': 136, 'break_even_price': '41.580000', 'high_price': '0.

Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data [{'chain_id': '769007c0-78c3-42ba-b275-e20da201a4cc', 'chain_symbol': 'HOOD', 'created_at': '2021-08-06T00:05:10.920641Z', 'expiration_date': '2021-09-03', 'id': '7e54d1f9-972e-46c5-a961-075f05368918', 'issue_date': '2021-08-06', 'min_ticks': {'above_tick': '0.10', 'below_tick': '0.05', 'cutoff_price': '3.00'}, 'rhs_tradability': 'untradable', 'state': 'active', 'strike_price': '53.0000', 'tradability': 'tradable', 'type': 'call', 'updated_at': '2021-08-06T00:05:10.920647Z', 'url': 'https://api.robinhood.com/options/instruments/7e54d1f9-972e-46c5-a961-075f05368918/', 'sellout_datetime': '2021-09-03T19:00:00+00:00', 'long_strategy_code': '7e54d1f9-972e-46c5-a961-075f05368918_L1', 'short_strategy_code': '7e54d1f9-972e-46c5-a961-075f05368918_S1', 'adjusted_mark_price': '1.100000', 'ask_price': '1.150000', 'ask_size': 12, 'bid_price': '1.050000', 'bid_size': 69, 'break_even_price': '54.10000

Loading Market Data Loading Market Data Loading Market Data Loading Market Data [{'chain_id': '79709730-7700-4372-b7e6-06a47c23830b', 'chain_symbol': 'LCID', 'created_at': '2021-08-17T02:02:45.448533Z', 'expiration_date': '2021-09-03', 'id': '55e0414c-f4ec-44e9-8f71-e47b276e5af5', 'issue_date': '2021-08-17', 'min_ticks': {'above_tick': '0.05', 'below_tick': '0.01', 'cutoff_price': '3.00'}, 'rhs_tradability': 'untradable', 'state': 'active', 'strike_price': '23.5000', 'tradability': 'tradable', 'type': 'call', 'updated_at': '2021-08-17T02:02:45.448539Z', 'url': 'https://api.robinhood.com/options/instruments/55e0414c-f4ec-44e9-8f71-e47b276e5af5/', 'sellout_datetime': '2021-09-03T19:00:00+00:00', 'long_strategy_code': '55e0414c-f4ec-44e9-8f71-e47b276e5af5_L1', 'short_strategy_code': '55e0414c-f4ec-44e9-8f71-e47b276e5af5_S1', 'adjusted_mark_price': '0.460000', 'ask_price': '0.490000', 'ask_size': 85, 'bid_price': '0.430000', 'bid_size': 29, 'break_even_price': '23.960000', 'high_price': '0

Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data [{'chain_id': 'eaa38263-3cde-4336-bf4f-1f0e2ef4824c', 'chain_symbol': 'GME', 'created_at': '2021-07-23T00:03:55.011435Z', 'expiration_date': '2021-09-03', 'id': '5c934e53-6b53-417a-bd99-b55de57ca96d', 'issue_date': '2021-07-23', 'min_ticks': {'above_tick': '0.05', 'below_tick': '0.01', 'cutoff_price': '3.00'}, 'rhs_tradability': 'untradable', 'state': 'active', 'strike_price': '250.0000', 'tradability': 'tradable', 'type': 'call', 'updated_at': '2021-07-23T00:03:55.011440Z', 'url': 'https://api.robinhood.com/options/instruments/5c934e53-6b53-417a-bd99-b55de57ca96d/', 'sellout_datetime': '2021-09-03T19:00:00+00:00', 'long_strategy_code': '5c934e53-6b53-417a-bd99-b55de57ca96d_L1', 'short_strategy_code': '5c934e53

...

Loading Market Data Loading Market Data Loading Market Data Loading Market Data [{'chain_id': '5868b3f8-d042-4bcf-ad66-45bc94d904f5', 'chain_symbol': 'LUV', 'created_at': '2021-07-22T00:07:46.281018Z', 'expiration_date': '2021-09-03', 'id': '30597a70-bfce-4737-80a1-b9312859b496', 'issue_date': '2021-07-22', 'min_ticks': {'above_tick': '0.05', 'below_tick': '0.01', 'cutoff_price': '3.00'}, 'rhs_tradability': 'untradable', 'state': 'active', 'strike_price': '53.0000', 'tradability': 'tradable', 'type': 'call', 'updated_at': '2021-07-22T00:07:46.281023Z', 'url': 'https://api.robinhood.com/options/instruments/30597a70-bfce-4737-80a1-b9312859b496/', 'sellout_datetime': '2021-09-03T19:00:00+00:00', 'long_strategy_code': '30597a70-bfce-4737-80a1-b9312859b496_L1', 'short_strategy_code': '30597a70-bfce-4737-80a1-b9312859b496_S1', 'adjusted_mark_price': '0.370000', 'ask_price': '0.400000', 'ask_size': 424, 'bid_price': '0.340000', 'bid_size': 196, 'break_even_price': '53.370000', 'high_price': '

Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data [{'chain_id': 'ff56bbe5-8652-4f5e-ba3b-d4652abf292a', 'chain_symbol': 'ARKK', 'created_at': '2021-07-22T00:07:35.856015Z', 'expiration_date': '2021-09-03', 'id': 'a7df6f54-360b-4209-b030-8a652fec0cd7', 'issue_date': '2021-07-22', 'min_ticks': {'above_tick': '0.05', 'below_tick': '0.01', 'cutoff_price': '3.00'}, 'rhs_tradability': 'untradable', 'state': 'active', 'strike_price': '124.5000', 'tradability': 'tradable', 'type': 'call', 'updated_at': '2021-07-22T00:07:35.856021Z', 'url': 'https://api.robinhood.com/options/instruments/a7df6f54-360b-4209-b030-8a652fec0cd7/', 'sellout_datetime': '2021-09-03T19:00:00+00:00', 'long_strategy_code': 'a7df6f54-360b-4209-b030-8a652fec0cd7_L1', 'short_strategy_code': 'a7df6f54-360b-4209-b030-8a652fec0cd7_S1', 'adjusted_mark_price': '0.850000', 'ask_price': '0.880000', 'ask_size': 7, 'bid_price': '0.820000', 'bid_

[]
excluded from screening because price is too high
[]
excluded from screening because price is too high
Loading Market Data Loading Market Data Loading Market Data Loading Market Data [{'chain_id': '368b7ddd-0f44-4ef0-a4d3-182b40953ed7', 'chain_symbol': 'RCL', 'created_at': '2021-07-29T00:10:16.327819Z', 'expiration_date': '2021-09-03', 'id': 'bbceae9d-0ad8-4bb6-a9e4-e0dd58a5dcd4', 'issue_date': '2021-07-29', 'min_ticks': {'above_tick': '0.05', 'below_tick': '0.01', 'cutoff_price': '3.00'}, 'rhs_tradability': 'untradable', 'state': 'active', 'strike_price': '86.0000', 'tradability': 'tradable', 'type': 'call', 'updated_at': '2021-07-29T00:10:16.327825Z', 'url': 'https://api.robinhood.com/options/instruments/bbceae9d-0ad8-4bb6-a9e4-e0dd58a5dcd4/', 'sellout_datetime': '2021-09-03T19:00:00+00:00', 'long_strategy_code': 'bbceae9d-0ad8-4bb6-a9e4-e0dd58a5dcd4_L1', 'short_strategy_code': 'bbceae9d-0ad8-4bb6-a9e4-e0dd58a5dcd4_S1', 'adjusted_mark_price': '0.850000', 'ask_price': '0.890000', '

Loading Market Data Loading Market Data [{'chain_id': 'd4a47316-3a3e-4ccd-83b7-ce4bea6a6064', 'chain_symbol': 'ET', 'created_at': '2021-07-22T00:07:58.207760Z', 'expiration_date': '2021-09-03', 'id': 'd20fc488-9b12-4ae1-be81-477243bd9cb3', 'issue_date': '2021-07-22', 'min_ticks': {'above_tick': '0.05', 'below_tick': '0.01', 'cutoff_price': '3.00'}, 'rhs_tradability': 'untradable', 'state': 'active', 'strike_price': '10.0000', 'tradability': 'tradable', 'type': 'call', 'updated_at': '2021-07-22T00:07:58.207765Z', 'url': 'https://api.robinhood.com/options/instruments/d20fc488-9b12-4ae1-be81-477243bd9cb3/', 'sellout_datetime': '2021-09-03T19:00:00+00:00', 'long_strategy_code': 'd20fc488-9b12-4ae1-be81-477243bd9cb3_L1', 'short_strategy_code': 'd20fc488-9b12-4ae1-be81-477243bd9cb3_S1', 'adjusted_mark_price': '0.040000', 'ask_price': '0.040000', 'ask_size': 535, 'bid_price': '0.030000', 'bid_size': 10, 'break_even_price': '10.040000', 'high_price': '0.050000', 'instrument': 'https://api.robi

Loading Market Data Loading Market Data Loading Market Data Loading Market Data [{'chain_id': 'b4a68a46-956c-4813-a16f-f28dfc3383a7', 'chain_symbol': 'FUBO', 'created_at': '2021-07-22T00:07:37.561816Z', 'expiration_date': '2021-09-03', 'id': '38d538e7-b839-469d-a6b1-ce5463aa06da', 'issue_date': '2021-07-22', 'min_ticks': {'above_tick': '0.05', 'below_tick': '0.01', 'cutoff_price': '3.00'}, 'rhs_tradability': 'untradable', 'state': 'active', 'strike_price': '29.0000', 'tradability': 'tradable', 'type': 'call', 'updated_at': '2021-07-22T00:07:37.561821Z', 'url': 'https://api.robinhood.com/options/instruments/38d538e7-b839-469d-a6b1-ce5463aa06da/', 'sellout_datetime': '2021-09-03T19:00:00+00:00', 'long_strategy_code': '38d538e7-b839-469d-a6b1-ce5463aa06da_L1', 'short_strategy_code': '38d538e7-b839-469d-a6b1-ce5463aa06da_S1', 'adjusted_mark_price': '0.450000', 'ask_price': '0.470000', 'ask_size': 547, 'bid_price': '0.430000', 'bid_size': 161, 'break_even_price': '29.450000', 'high_price': 